In [3]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
import json
import os
from typing import Annotated, TypedDict, List, Literal
from dotenv import load_dotenv

# RAG 관련 모듈 추가
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader, TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import Annotated, List


from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
import json
import os
import random
from typing import Annotated, TypedDict, List, Literal
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage , SystemMessage
from langchain_teddynote.graphs import visualize_graph
from langchain_core.runnables import RunnableLambda, RunnableMap, RunnableConfig
from langchain_core.prompts import ChatPromptTemplate
import ast
from IPython.display import Image, display


# RAG 관련 모듈 추가
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader, TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.runnables import RunnableConfig
from langgraph.checkpoint.memory import MemorySaver
from typing import Annotated, List
from datetime import datetime
from langchain_core.output_parsers import PydanticOutputParser
from typing import Literal, get_args

# # src 내 모듈 import
# import ModelManager
# import PromptTemplate
# import RagManager
# import Structure

# ## RAG 문서 경로
# docs_path = os.path.join(os.path.dirname(__file__), '..', 'docs')
# docs_path = os.path.abspath(docs_path)

In [4]:
## 문서 종류에 따라 조건분기 함수 
def docu_routing(state):
    
    ''' 문서 종류에 따라 조건분기 함수 '''
    
    file_path = state.get('file_path' , None)       # 파일 경로
    
    if file_path == None: return 'None'
    
    file_ext = file_path.split('.')[-1]             # 파일 확장자
    
    # pdf , txt , pptx 파일만 처리
    if file_ext in ['pdf' , 'txt' , 'pptx']:
        return file_ext
    
    # 다른 파일 형식은 None 반환 (RAG 적용 x)
    return 'None'


# pdf 파일 로더
def load_pdf_docu(state):
    
    """ PDF 파일 로더 """
    
    file_path = state.get('file_path')
    return {'input_docu' : load_pdf_docu_path(file_path)}


def load_pdf_docu_path(file_path):
    
    """ PDF 파일 로더 """
    
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    
    return documents


# txt 파일 로더 >> 
def load_txt_docu(state):
    
    """ TXT 파일 로더 """
    
    file_path = state.get('file_path')
    
    try:
        loader = TextLoader(file_path, encoding='utf-8')
        docu = loader.load() 
        return {'input_docu' : loader.load() }
    
    except UnicodeDecodeError:
        loader = TextLoader(file_path, encoding='cp949')  # 한글 윈도우 인코딩
        return {'input_docu' : loader.load() }


# pptx 파일 로더
def load_pptx_docu(state):
    
    """ PPTX 파일 로더 """
    pass




class RAGManager:
    
    def __init__(self, docs_dir):
        
        self.embeddings = OpenAIEmbeddings()
        self.vector_store = None
        
    def load_and_process_documents(self , doc):
        
        """문서를 로드하고 벡터화하여 저장"""
        
        # 텍스트 분할기
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )

        # Chunk 분할
        chunks = text_splitter.split_documents(doc)
        
        # 벡터 저장소 생성
        self.vector_store = Chroma.from_documents(documents=chunks, embedding=self.embeddings)
        
        return len(chunks)

    # 검색 
    def get_relevant_documents(self, query, k=3):
        
        """쿼리에 관련된 문서 검색"""
        if not self.vector_store:
            raise ValueError("벡터 저장소가 초기화되지 않았습니다. load_and_process_documents()를 먼저 호출하세요.")
        
        docs = self.vector_store.similarity_search(query, k=k)
        return docs
    
    def get_context_str(self, docs):
        """검색된 문서를 문맥 문자열로 변환"""
        context = "\n\n".join([doc.page_content for doc in docs])
        return context


def rag_2(input_docu , query):
    
    rag_manager = RAGManager()
    rag_manager.load_and_process_documents(input_docu)
    relevant_docs = rag_manager.get_relevant_documents(query)
    
    print("="*100)
    print('RAG 쿼리 결과 : ')
    for doc in relevant_docs:
        print(doc.page_content)
    
    context = rag_manager.get_context_str(relevant_docs)
    
    return context


def rag_query(state):
    
    context = ""
    input_docu = state['input_docu']
    query = state.get('query' , '')
    
    match state['rag_option']:
        case 0:
            pass
        case 1:
            context = input_docu[0].page_content 
        case 2:
            context = rag_2(input_docu , query)
        case 3:
            pass
     
    return {'context' : context}
  



def rag_module():
    
    # StateGraph 생성
    builder = StateGraph(Structure.ragState)

    ## 노드 설정 ##
    builder.add_node( "Load_PDF" , load_pdf_docu)        
    builder.add_node( 'Load_TXT' , load_txt_docu)      
    builder.add_node( 'Load_PPT' , load_pptx_docu)      
    builder.add_node( 'RAG' , rag_query)           
         

    # 엣지 설정 ##
    builder.add_edge('Load_PDF', 'RAG')
    builder.add_edge('Load_TXT', 'RAG')
    builder.add_edge('Load_PPT', 'RAG')
    builder.add_edge('RAG', END)
    
    ## 조건부 엣지 설정 ##
    builder.add_conditional_edges( START, docu_routing ,
        { 'pdf' : 'Load_PDF',
           'txt' : 'Load_TXT',
           'pptx' : 'Load_PPT' , 'None' : END }
    )


    # ## 조건부 엣지 설정 ##
    # builder.add_conditional_edges( START, docu_routing ,
    #     { 'pdf' : END}
    # )
    
    # 그래프 컴파일
    app = builder.compile(checkpointer = MemorySaver())
    
    img_txt = app.get_graph().draw_mermaid()
    print(img_txt)
    # with open("RAG_Graph.png", "wb") as f:
    #     f.write(img.data)
        
    return app


def exec_rag(input_dict):
    
    RAG_graph = rag_module()
    
    # Config 설정
    config = RunnableConfig(
        recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
        configurable={"thread_id": "7"},  # 스레드 ID 설정
        tags=["my-tag"],  # Tag
    )

    event_list = []
    
    # Graph 실행
    for event in RAG_graph.stream(input =  input_dict, config=config):
        event_list.append(event)
        
    return event_list

In [5]:
### 2). PDF 로더 테스트 ###
pdf_file_path_1 = '../docs/2024 내일은 빅데이터분석기사 필기 핵심 요약집.pdf'
pdf_file_path_2 = '../docs/온디바이스 AI 기술동향 및 발전방향.pdf'

docu = load_pdf_docu_path(pdf_file_path_2)
docu

[Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.4 (Windows)', 'creationdate': '2024-06-26T17:50:32+09:00', 'moddate': '2024-06-27T22:55:19+09:00', 'trapped': '/False', 'source': '../docs/온디바이스 AI 기술동향 및 발전방향.pdf', 'total_pages': 39, 'page': 0, 'page_label': '1'}, page_content='온디바이스 AI \n기술동향 및 발전방향\nISSUE \nREPORT \n2024-06호'),
 Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.4 (Windows)', 'creationdate': '2024-06-26T17:50:32+09:00', 'moddate': '2024-06-27T22:55:19+09:00', 'trapped': '/False', 'source': '../docs/온디바이스 AI 기술동향 및 발전방향.pdf', 'total_pages': 39, 'page': 1, 'page_label': '2'}, page_content='ISSUE \nREPORT \n2024-06호\nDIGISIGHT 온디바이스 AI 기술동향 및 발전방향\nⅠ. 개 요 06\nⅡ. 시장 및 기업 동향 09\nⅢ. 핵심기술 동향 12\nⅣ. 미래 발전방향 20 \n 27\n 30\n 33\n 35\nKEA NOW\nMEMBER NEWS\nESG TREND\nSTATS'),
 Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.4 (Windows)', 'creationdate': '2024-06-26T

In [23]:
docu[0].metadata

{'producer': 'Adobe PDF Library 17.0',
 'creator': 'Adobe InDesign 19.4 (Windows)',
 'creationdate': '2024-06-26T17:50:32+09:00',
 'moddate': '2024-06-27T22:55:19+09:00',
 'trapped': '/False',
 'source': '../docs/온디바이스 AI 기술동향 및 발전방향.pdf',
 'total_pages': 39,
 'page': 0,
 'page_label': '1'}

In [12]:
docu[1].page_content

'ISSUE \nREPORT \n2024-06호\nDIGISIGHT 온디바이스 AI 기술동향 및 발전방향\nⅠ. 개 요 06\nⅡ. 시장 및 기업 동향 09\nⅢ. 핵심기술 동향 12\nⅣ. 미래 발전방향 20 \n 27\n 30\n 33\n 35\nKEA NOW\nMEMBER NEWS\nESG TREND\nSTATS'

In [8]:
docu[1]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.4 (Windows)', 'creationdate': '2024-06-26T17:50:32+09:00', 'moddate': '2024-06-27T22:55:19+09:00', 'trapped': '/False', 'source': '../docs/온디바이스 AI 기술동향 및 발전방향.pdf', 'total_pages': 39, 'page': 1, 'page_label': '2'}, page_content='ISSUE \nREPORT \n2024-06호\nDIGISIGHT 온디바이스 AI 기술동향 및 발전방향\nⅠ. 개 요 06\nⅡ. 시장 및 기업 동향 09\nⅢ. 핵심기술 동향 12\nⅣ. 미래 발전방향 20 \n 27\n 30\n 33\n 35\nKEA NOW\nMEMBER NEWS\nESG TREND\nSTATS')

In [15]:
print(docu[2].page_content)

DIGISIGHT
   2024.06  제6호
Ⅰ. 개 요
Ⅱ. 시장 및 기업 동향 
Ⅲ. 핵심기술 동향 
Ⅳ. 미래 발전방향
  참고문헌 
 
온디바이스 AI 
기술동향 및 발전방향


In [18]:
for page in range(docu[0].metadata['total_pages']):
    print()
    print(docu[page].page_content)


온디바이스 AI 
기술동향 및 발전방향
ISSUE 
REPORT 
2024-06호

ISSUE 
REPORT 
2024-06호
DIGISIGHT 온디바이스 AI 기술동향 및 발전방향
Ⅰ. 개 요 06
Ⅱ. 시장 및 기업 동향 09
Ⅲ. 핵심기술 동향 12
Ⅳ. 미래 발전방향 20 
 27
 30
 33
 35
KEA NOW
MEMBER NEWS
ESG TREND
STATS

DIGISIGHT
   2024.06  제6호
Ⅰ. 개 요
Ⅱ. 시장 및 기업 동향 
Ⅲ. 핵심기술 동향 
Ⅳ. 미래 발전방향
  참고문헌 
 
온디바이스 AI 
기술동향 및 발전방향

요 약
   (정의) 온디바이스 AI(On-Device AI)란 데이터를 외부 서버나 클라우드에 전송하지 
않고  디바이스 자체적으로 AI연산을 수행하는 기술
◎  대용량데이터처리제한,개인정보유출위험,실시간성저하등기존서버기반중앙집중형구조의여러
제약사항을해결하기위해등장
◎  높은컴퓨팅파워가필요한AI연산을다양한하드웨어및소프트웨어기술을활용하여최적화된스마트폰,
로봇,드론등디바이스내에서AI연산수행
◎  디바이스내자체AI연산을통해네트워크환경에독립적인실시간서비스,개인정보유출최소화,서버
운영비용절감등강점
◎  NPU,AI모델최적화등고수준의하드웨어및소프트웨어기술동시요구로인해높은진입장벽존재
  (시장동향) 글로벌 기업 및 각국 정부 투자 가속화로 고성장 예상 
◎  생성형AI및온디바이스AI시장은전세계적으로빅테크기업투자열풍으로급성장이예상되며,각국
정부들도투자계획수립을서두르고있어향후시장확대가가속화될전망
◎  미국을포함한유럽,일본,중국,대만,한국등주요국가에서디바이스중심의AI관련기술개발지원정책을
지속적으로발표
◎  AI칩제조사들은자사칩기반생태계구축을위해데이터·AI모델·추론·SDK등전방위적기술

In [ ]:
if __name__ == "__main__":
    
    ## .env 파일 로드
    load_dotenv('../.env')


    ### 1). TXT 로더 테스트 ###
    txt_file_path = '../docs/빅데이터분석기사.txt'
    # test_docu = load_txt_docu(test_file_path)
    # print(test_docu[0].page_content)
    
    
    ### 2). PDF 로더 테스트 ###
    pdf_file_path_1 = '../docs/2024 내일은 빅데이터분석기사 필기 핵심 요약집.pdf'
    pdf_file_path_2 = '../docs/온디바이스 AI 기술동향 및 발전방향.pdf'
    
    print(load_pdf_docu_path(pdf_file_path_2))
    
    import json
    json.dump(load_pdf_docu_path(pdf_file_path_2) , open('test.json' , 'w' , encoding='utf-8') )
    
    
    ### 3). RAG 옵션1 테스트 ###
    # input_dict = {'file_path' : txt_file_path  , 'rag_option' : 2 , 'query' : '데이터 전처리'}
    # exec_rag(input_dict)



    ### 4). RAG 옵션2 테스트 ###
    

    
    
    # # RAG 매니저 인스턴스 생성
    # rag_manager = RAGManager()
    
    # # 문서 로드 및 처리
    # num_docs = rag_manager.load_and_process_documents()
    # print(f"Loaded {num_docs} documents.")
    
    # # 쿼리 검색
    # query = "ADsP 통계"
    # docs = rag_manager.get_relevant_documents(query)
    
    # # 문맥 문자열 생성
    # context_str = rag_manager.get_context_str(docs)
    # print("Context String:")
    # print(context_str)  
    
    